In [ ]:
import os

from qiskit.transpiler.passes.optimization import *
from qiskit.transpiler.passes import *
from qiskit.circuit.equivalence_library import SessionEquivalenceLibrary

from pytket import OpType
from pytket.passes import *

from FT_optimization_generate_data import *

SINGLE_QUBIT_AND_CX_QISKIT_STDGATES = [
    "x",
    "y",
    "z",
    "h",
    "s",
    "sdg",
    "t",
    "tdg",
    "sx",
    "rx",
    "ry",
    "rz",
    "cx",
]

SINGLE_QUBIT_AND_CX_TKET_STDGATES = {
    OpType.Rx,
    OpType.Ry,
    OpType.Rz,
    OpType.X,
    OpType.Y,
    OpType.Z,
    OpType.H,
    OpType.S,
    OpType.Sdg,
    OpType.T,
    OpType.Tdg,
    OpType.SX,
    OpType.CX
}

# Qiskit

In [ ]:
circuit_folder = "MQTBenchQiskit"

benchmarks = [f.replace(".qasm", "") for f in os.listdir(circuit_folder) if f.endswith(".qasm")]
benchmark_names = [name.split("_indep_qiskit_")[0] for name in benchmarks]
benchmarks.sort()

basis_gates = SINGLE_QUBIT_AND_CX_QISKIT_STDGATES

transpiler_passes = [
    [Optimize1qGatesDecomposition(basis=basis_gates), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
    [Optimize1qGatesSimpleCommutation(basis=basis_gates), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
    [CommutationAnalysis(), CommutativeCancellation(basis_gates=basis_gates), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
    [HoareOptimizer(), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
    [TemplateOptimization(), BasisTranslator(SessionEquivalenceLibrary, basis_gates)],
]

In [ ]:
generate_data(
    f"results_qiskit.xlsx",
    benchmarks, transpiler_passes,
    None,
    'qiskit',
)

# Tket

In [ ]:
circuit_folder = "MQTBenchTKET"

benchmarks = [f.replace(".qasm", "") for f in os.listdir(circuit_folder) if f.endswith(".qasm")]
benchmark_names = [name.split("_indep_tket_")[0] for name in benchmarks]
benchmarks.sort()

transpiler_passes = [
    RemoveRedundancies(),
    CliffordSimp(),
    FullPeepholeOptimise(),
    EulerAngleReduction(OpType.Rx, OpType.Ry),
    KAKDecomposition(),
    RepeatPass(SequencePass([CommuteThroughMultis(), RemoveRedundancies()]))
]

transpiler_pass_names = [
    "RemoveRedundancies",
    "CliffordSimp",
    "FullPeepholeOptimise",
    "EulerAngleReduction",
    "KAKDecomposition",
    "RepeatPass"
]

In [ ]:
generate_data(
    f"results_tket.xlsx",
    benchmarks,
    transpiler_passes,
    transpiler_pass_names,
    'tket'
)